In [ ]:
import xml.dom.minidom as minidom
import xml.etree.ElementTree as xmlET

import pandas as pd
import numpy as np

from future.utils import iteritems
from pyPRMS.prms_helpers import float_to_str

In [ ]:
# Manually generated control file parameters from setup.c
# workfile = '/Users/pnorton/Projects/National_Hydrology_Model/documents/PRMS6/prms_params_raw.txt'
df = pd.read_csv(workfile, sep='\t')

df.head()

### Generate a XML file for parameters

In [ ]:
#     <parameter name="ssstor_init_frac">
#         <type>F</type>
#         <units>decimal fraction</units>
#         <desc>Initial fraction of available water in the gravity plus preferential-flow reservoirs</desc>
#         <help>Initial fraction of available water in the gravity plus preferential-flow reservoirs (fraction of sat_threshold) for each HRU</help>
#         <minimum>0.0</minimum>
#         <maximum>1.0</maximum>
#         <default>0.0</default>
#         <modules>
#             <module>soilzone</module>
#         </modules>
#     </parameter>

NHM_DATATYPES = {'I': 'integer', 'F': 'real', 'D': 'double', 'S': 'string'}
inv_map = {vv: kk for kk, vv in iteritems(NHM_DATATYPES)}

params_xml = xmlET.Element('parameters')

for index, row in df.iterrows():
#     print(row)
    param_sub = xmlET.SubElement(params_xml, 'parameter')
    param_sub.set('name', row['parameter'])

    xmlET.SubElement(param_sub, 'type').text = inv_map[row['datatype']]

    xmlET.SubElement(param_sub, 'units').text = row['units']
    xmlET.SubElement(param_sub, 'desc').text = row['description']
    xmlET.SubElement(param_sub, 'help').text = row['help']
    
    if isinstance(row['min_value'], basestring):
        xmlET.SubElement(param_sub, 'minimum').text = row['min_value']
    else:
        if row['datatype'] in ['real', 'double']:
            xmlET.SubElement(param_sub, 'minimum').text = float_to_str(row['min_value'])
        elif row['datatype'] == 'integer':
            xmlET.SubElement(param_sub, 'minimum').text = str(int(row['min_value']))  
        
    if isinstance(row['max_value'], basestring):
        xmlET.SubElement(param_sub, 'maximum').text = row['max_value']
    else:
        if row['datatype'] in ['real', 'double']:
            xmlET.SubElement(param_sub, 'maximum').text = float_to_str(row['max_value'])
        elif row['datatype'] == 'integer':
            xmlET.SubElement(param_sub, 'maximum').text = str(int(row['max_value']))
            
    if isinstance(row['default_value'], basestring):
        xmlET.SubElement(param_sub, 'default').text = row['default_value']
    else:
        if row['datatype'] in ['real', 'double']:
            xmlET.SubElement(param_sub, 'default').text = float_to_str(row['default_value'])
        elif row['datatype'] == 'integer':
            xmlET.SubElement(param_sub, 'default').text = str(int(row['default_value']))
            
    modules_sub = xmlET.SubElement(param_sub, 'modules')

    for mm in (row['modules']).split(','):
        xmlET.SubElement(modules_sub, 'module').text = mm
        
    dimensions_sub = xmlET.SubElement(param_sub, 'dimensions')
    
    for ii, dd in enumerate((row['dimensions']).split(',')):
        curr_dim = xmlET.SubElement(dimensions_sub, 'dimension')
        curr_dim.set('name', dd)
        xmlET.SubElement(curr_dim, 'position').text = str(ii+1)
 

In [ ]:
xmlstr = minidom.parseString(xmlET.tostring(params_xml)).toprettyxml(indent='    ')

with open('<path>/tmp/parameters.xml', 'w') as ff:
    ff.write(xmlstr)